In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder

In [ ]:
path = '/Users/tanlinyi/Downloads/'
train_file = 'aug_train.csv'
test_file = 'aug_test.csv'

In [ ]:
df = pd.read_csv(path + train_file)
df_raw = df.copy()

In [ ]:
df.columns

In [ ]:
df['target'].value_counts()

In [ ]:
df.shape

In [ ]:
for col in df.columns:
    print(f'Column: {col}')
    print(f'Data Type: {df[col].dtype}')
    print(f'Number of NaN: {df[col].isnull().sum()}')
    print(f'Number of Unique Values: {df[col].nunique()}')
    print(f'Number of Unique Values: {df[col].unique()}')
    print('')

### Data Preprocessing: NaN Values

In [ ]:
missing_value_counts = pd.DataFrame(df.isnull().sum(axis=1))
missing_value_counts.columns = ['count']
missing_value_counts['count'].value_counts()

In [ ]:
df.dropna(axis=0, thresh=10, inplace=True) # Dropping Rows where Null Counts > 4
print(f" Original Numbers of Rows: {df.shape[0]} Current Numbers of Rows: {df.shape[0]}")

### Preprocessing: City

In [ ]:
df['city'].value_counts(dropna=False)

In [ ]:
city_dict = df['city'].value_counts().to_dict()

In [ ]:
print(df['city'].nunique())

In [ ]:
values = list(city_dict.values())
q1 = np.percentile(values, 25)
median = np.median(values)
q3 = np.percentile(values, 75)

size_bins_cities = {'small': [], 'medium': [], 'large': [], 'very_large': []}
for city, count in city_dict.items():
    if count <= q1:
        size_bins_cities['small'].append(city)
    if q1 < count <= median:
        size_bins_cities['medium'].append(city)
    if median < count <= q3:
        size_bins_cities['large'].append(city)
    else:
        size_bins_cities['very_large'].append(city)

In [ ]:
df.loc[df['city'].isin(size_bins_cities['small']), 'city'] = 'city_small'
df.loc[df['city'].isin(size_bins_cities['medium']),  'city'] = 'city_medium'
df.loc[df['city'].isin(size_bins_cities['large']), 'city'] = 'city_large'
df.loc[df['city'].isin(size_bins_cities['very_large']),  'city'] = 'city_xlarge'

In [ ]:
df['city'].value_counts()

In [197]:
# Label Encoding `city` into 4 categories
le_city = LabelEncoder()
df['city'] = le_city.fit_transform(df['city'])
dict(zip(le_city.classes_, le_city.transform(le_city.classes_)))

{'city_large': np.int64(0),
 'city_medium': np.int64(1),
 'city_small': np.int64(2),
 'city_xlarge': np.int64(3)}

### Preprocessing: `gender`

In [201]:
df['gender'].value_counts(dropna=False)

gender
Male      13185
NaN        4300
Female     1237
Other       186
Name: count, dtype: int64

In [202]:
df['gender'].fillna('Other')

0         Male
1         Male
2        Other
3        Other
4         Male
         ...  
19153     Male
19154     Male
19155     Male
19156     Male
19157    Other
Name: gender, Length: 18908, dtype: object

In [203]:
gender_ohe = pd.get_dummies(df['gender'], prefix = 'gender')
df = df.loc[:, df.columns != 'gender'].join(gender_ohe)

### Preprocessing: `relevent_experience`

In [206]:
df.rename(columns={'relevent_experience': 'relevant_experience'}, inplace=True)

In [207]:
df['relevant_experience'].value_counts(dropna=False)

relevant_experience
Has relevent experience    13734
No relevent experience      5174
Name: count, dtype: int64

In [208]:
le_re = LabelEncoder()
df['relevant_experience'] = le_re.fit_transform(df['relevant_experience'])
dict(zip(le_re.classes_, le_re.transform(le_re.classes_)))

{'Has relevent experience': np.int64(0), 'No relevent experience': np.int64(1)}

### Preprocessing: `enrolled_university`

In [211]:
df['enrolled_university'].value_counts(dropna=False)

enrolled_university
no_enrollment       13729
Full time course     3719
Part time course     1189
NaN                   271
Name: count, dtype: int64

In [212]:
df['enrolled_university'] = df['enrolled_university'].fillna('Unknown')

In [213]:
le = LabelEncoder()
df['enrolled_university'] = le.fit_transform(df['enrolled_university'])
dict(zip(le.classes_, le.transform(le.classes_)))

{'Full time course': np.int64(0),
 'Part time course': np.int64(1),
 'Unknown': np.int64(2),
 'no_enrollment': np.int64(3)}

### Preprocessing: `education_level`

In [219]:
df['education_level'].value_counts(dropna=False)

education_level
Graduate          11591
Masters            4357
High School        1963
Phd                 413
Primary School      300
NaN                 284
Name: count, dtype: int64

In [220]:
df['education_level'] = df['education_level'].fillna('Unknown')

In [ ]:
le_el = LabelEncoder()
df['education_level'] = le_el.fit_transform(df['education_level'])
dict(zip(le_el.classes_, le_el.transform(le_el.classes_)))

### Preprocessing: `major_discipline`

In [224]:
df['major_discipline'].value_counts(dropna = False)

major_discipline
STEM               14483
NaN                 2572
Humanities           669
Other                381
Business Degree      327
Arts                 253
No Major             223
Name: count, dtype: int64

In [225]:
df['major_discipline']= df['major_discipline'].fillna('Unknown')

In [226]:
le_md = LabelEncoder()
df['major_discipline'] = le_md.fit_transform(df['major_discipline'])
dict(zip(le_md.classes_, le_md.transform(le_md.classes_)))

{'Arts': np.int64(0),
 'Business Degree': np.int64(1),
 'Humanities': np.int64(2),
 'No Major': np.int64(3),
 'Other': np.int64(4),
 'STEM': np.int64(5),
 'Unknown': np.int64(6)}

### Preprocessing: `company_size`

In [230]:
df['company_size'].value_counts(dropna=False)

company_size
NaN          5695
50-99        3083
100-500      2568
10000+       2018
10/49        1469
1000-4999    1328
<10          1307
500-999       877
5000-9999     563
Name: count, dtype: int64

In [231]:
df['company_size'].mode()[0]

'50-99'

In [232]:
df['company_size'] = df['company_size'].fillna('Small') # Purely Because

In [234]:
df['company_size'] = df['company_size'].replace(
    ['<10', '10/49', '50-99', '100-500', '500-999', '1000-4999', '5000-9999', '10000+'],
    ['Startup', 'Small', 'Small', 'Medium', 'Medium', 'Large', 'Large', 'Large'])

In [235]:
le_cs = LabelEncoder()
df['company_size'] = le_cs.fit_transform(df['company_size'])
dict(zip(le_cs.classes_, le_cs.transform(le_cs.classes_)))

{'Large': np.int64(0),
 'Medium': np.int64(1),
 'Small': np.int64(2),
 'Startup': np.int64(3)}

### Preprocessing: `company_type`

In [247]:
df['company_type'].value_counts(dropna = False, ascending = True)

company_type
Other                   121
NGO                     521
Early Stage Startup     603
Public Sector           954
Funded Startup         1001
NaN                    5894
Pvt Ltd                9814
Name: count, dtype: int64

In [248]:
df['company_type'] = df['company_type'].fillna('Unknown')

In [249]:
le_ct = LabelEncoder()
df['company_type'] = le_ct.fit_transform(df['company_type'])
dict(zip(le_ct.classes_, le_ct.transform(le_ct.classes_)))

{'Early Stage Startup': np.int64(0),
 'Funded Startup': np.int64(1),
 'NGO': np.int64(2),
 'Other': np.int64(3),
 'Public Sector': np.int64(4),
 'Pvt Ltd': np.int64(5),
 'Unknown': np.int64(6)}

### Preprocessing: `last_new_job`

In [238]:
df['last_new_job'].value_counts(dropna=False)

last_new_job
1        8004
>4       3279
2        2890
never    2351
4        1028
3        1021
NaN       335
Name: count, dtype: int64

In [239]:
df['last_new_job'] = df['last_new_job'].replace(['>4', 'never'], [4, 0])
df['last_new_job'] = df['last_new_job'].fillna(df['last_new_job'].mode()[0])
df['last_new_job'] = [float(i) for i in df['last_new_job']]

### Preprocessing: `training_hours`

In [244]:
df['training_hours'].isnull().sum()

np.int64(0)

### Preprocessing `experience`
- Notice that we will produce one as an input feature for the binary classification, and another as the target non-binary classification

In [252]:
df['experience'].value_counts(dropna=False)

experience
>20    3269
5      1411
4      1381
3      1321
6      1204
2      1080
7      1023
10      985
9       976
8       795
15      683
11      662
14      586
1       526
16      505
<1      498
12      491
13      398
17      341
19      303
18      278
20      146
NaN      46
Name: count, dtype: int64

In [254]:
df['experience'] = df['experience'].replace(['<1', '>20'], [1, 20])
df['experience'] = df['experience'].fillna(0)
df['experience'] = df['experience'].astype(int)

In [261]:
# Preprocessing for Target Variable
df['experience_temp'] = df['experience'].copy()
junior_mask = df['experience_temp'] <= 3
mid_mask = (df['experience_temp'] > 3) & (df['experience_temp'] <= 12)
senior_mask = df['experience_temp'] >= 12

In [262]:
df['seniority_level'] = np.select([junior_mask, mid_mask, senior_mask], [0, 1, 2]).astype(int)

In [263]:
df['seniority_level'].value_counts(dropna=False)

seniority_level
1    8928
2    6509
0    3471
Name: count, dtype: int64

### Forming Databases

In [266]:
df.columns

Index(['enrollee_id', 'city', 'city_development_index', 'relevant_experience',
       'enrolled_university', 'education_level', 'major_discipline',
       'experience', 'company_size', 'company_type', 'last_new_job',
       'training_hours', 'target', 'gender_Female', 'gender_Male',
       'gender_Other', 'experience_temp', 'seniority_level'],
      dtype='object')

In [275]:
df_binary = df.copy()
df_binary = df_binary.drop(columns =['experience_temp', 'seniority_level'])

In [276]:
df_binary['target'] = df_binary['target'].astype(int)

In [277]:
cols = [c for c in df_binary.columns if c != 'target'] + ['target']
df_binary = df_binary[cols]

In [279]:
df_binary.to_csv(path + 'Binary Database_Tan.csv', index=False)

In [282]:
# Non-Binary Classification
df_multi = df.copy()
df_multi = df_multi.drop(columns = ['experience', 'experience_temp', 'target'])

In [284]:
df_multi.to_csv(path + 'Non-Binary Database_Tan.csv', index=False)

### Visualizing the Variables

In [ ]:
categorical_col = ['city', 'gender',
                   'relevent_experience', 'enrolled_university', 'education_level',
                   'major_discipline', 'experience', 'company_size', 'company_type',
                   'last_new_job', 'target']
numerical_col = ['city_development_index', 'training_hours']

In [ ]:
plt.style.use('seaborn-v0_8')

In [ ]:
# Boxplot for Categorical Variables
for i in range(0, len(categorical_col), 4):
    subset = categorical_col[i:i + 4]  # take 4 columns per figure

    plt.figure(figsize=(14, 10))

    for j, col in enumerate(subset, 1):
        plt.subplot(2, 2, j)

        counts = df[col].value_counts()
        plt.bar(counts.index.astype(str), counts.values)
        plt.title(f'Bar Chart for {col}')

    plt.tight_layout()
    plt.savefig(f'{path} Graph_{i}.png')
    plt.show()

In [ ]:
# Numerical Values
plt.figure()
plt.subplot(1, 2, 1)
plt.boxplot(df[numerical_col[0]])
plt.title('Box Plot for city_development_index')
plt.ylabel('city_development_index')
plt.subplot(1, 2, 2)
plt.boxplot(df[numerical_col[1]])
plt.title('Box Plot for training_hours')
plt.ylabel('training_hours')
plt.savefig(f'{path} Graph_4.png')
plt.show()

In [285]:
df_multi.columns

Index(['enrollee_id', 'city', 'city_development_index', 'relevant_experience',
       'enrolled_university', 'education_level', 'major_discipline',
       'company_size', 'company_type', 'last_new_job', 'training_hours',
       'gender_Female', 'gender_Male', 'gender_Other', 'seniority_level'],
      dtype='object')